
<a name="top" id="top"></a>

  <div align="center">
    <h1>Max Flow</h1>
   <a href="https://github.com/abdelane">Abigail B. Delaney</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <i>Undergraduate Research Assistant</i>
    <br>
    <br>
    <a href="https://github.com/bernalde">David E. Bernal Neira</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <br>
    <a href = "https://github.com/Pyomo/PyomoGallery/tree/master/maxflow">
        <img src="https://img.shields.io/badge/-PyomoGallery-blue" alt="PyomoGallery"/>
        </a>
    <br>
    <a href="https://colab.research.google.com/github/abdelane/pyomogallery/blob/main/maxflowAB/maxflowAB.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
    <a href="https://secquoia.github.io/">
        <img src="https://img.shields.io/badge/🌲⚛️🌐-SECQUOIA-blue" alt="SECQUOIA"/>
    </a>
</div>

# Max Flow

## Summary

The goal of the maximum flow problem is to find the maximum flow possible in a network from some given source node to a given sink node.  Applications of the max flow problem include finding the maximum flow of orders through a job shop, the maximum flow of water through a storm sewer system, and the maximum flow of product through a product distribution system, among others. Schrijver (2002) note that the maximum flow problem was first formulated in 1954 by T. E. Harris and F. S. Ross as a simplified model of Soviet railway traffic flow.

A network is a directed graph, and the arc capacities, or upper bounds, are the only relevant parameters.  A network graph does not have to be symmetric: if an arc (v,w) is in the graph, the reverse arc (w,v) does not have to be in the graph. Further, parallel arcs are not allowed, and self-loops are not allowed. The source and the sink are distinct nodes in the network, but the sink may be unreachable from the source.
 

We begin by importing the Pyomo package and other necessary functionality to run on local computer or google colab.

In [ ]:
# If using this on Google collab, we need to install the packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Let's start with Pyomo
if IN_COLAB:
    !pip install -q pyomo

#uploading other files from computer to colab
if IN_COLAB:
  !wget https://raw.githubusercontent.com/Pyomo/PyomoGallery/refs/heads/master/maxflow/maxflow.dat

# Let's install the LP/MIP solver GLPK
if IN_COLAB:
    !apt-get install -y -qq glpk-utils
    !pip install git+https://github.com/bruscalia/gethighs


## Problem Statement

The max flow problem can be formulated mathematically as a linear programming problem using the following model.

## Pyomo Formulation

We begin by importing the Pyomo package and other necessary functionality to run on local computer or google colab. An abstract model is used to allow for the code to be repurposed for other data sets.

In [ ]:
#imports pyomo package
import pyomo.environ as pyo
#imports solver HiGHS
from gethighs import HiGHS

# Define the solver GLPK and HiGHS
if IN_COLAB:
    opt_glpk = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
  
else:
    opt_glpk = pyo.SolverFactory('glpk')
    opt_HiGHS = HiGHS()
# Here we could use another solver, e.g. gurobi or cplex
# opt_gurobi = SolverFactory('gurobi')


#sets infinity to a float value
infinity = float('inf')

#creates an abstract model named model
model = pyo.AbstractModel()

### Sets

 $N$ = nodes in the network  
 $A$ = network arcs

The sets $N$ and $A$ are declared abstractly using the `Set` component:

In [ ]:
# Nodes in the network
# creates an empty set of nodes in the network to be filled later
model.N = pyo.Set(doc = 'nodes')

# Network arcs
# creates an empty set of arcs in the network which connect two nodes (there are N^2 arcs)
model.A = pyo.Set(within=model.N*model.N, doc = 'arcs')

### Parameters

 $s$ = source node  
 $t$ = sink node  
 $c_{ij}$ = arc flow capacity, $\forall (i,j) \in A$
 
Similarly, the model parameters are defined abstractly using the `Param` component:

In [ ]:
# Source node
# origin of flow and a parameter within the set of nodes
model.s = pyo.Param(within=model.N, doc = 'source node')

# Sink node
# end of flow and a parameter within the set of nodes
model.t = pyo.Param(within=model.N, doc = 'sink node')

# Flow capacity limits
# upper bound of flow for each arc
model.c = pyo.Param(model.A, doc= 'flow capacity')

### Variables
 $f_{i,j}$ = arc flow, $\forall (i,j) \in A$

The `within` option is used in these parameter declarations to define expected properties of the parameters.  This information is used to perform error checks on the data that is used to initialize the parameter components.

The `Var` component is used to define the decision variables:

In [ ]:
# The flow over each arc
# variable for flow over each arc which is greater than zero (accounting for later constraints)
model.f = pyo.Var(model.A, domain=pyo.NonNegativeReals, doc = 'flow over arc')

### Objective

Maximize the flow into the sink nodes  
 $\max \sum_{\{i \mid (i,t) \in A\}} c_{i,t} f_{i,t}$

The `within` option is used to restrict the domain of the decision variables to the non-negative reals. This eliminates the need for explicit bound constraints for variables.

The `Objective` component is used to define the cost objective.  This component uses a rule function to construct the objective expression:

In [ ]:
# Maximize the flow into the sink nodes
def total_rule(model):
    '''
    Function to sum the arc capacity times the flow over each arc
    
    Parameters
    ----------
    model (AbstractModel): The model for which the objctive function to solve
    
    Returns
    -------
    summation: The sum of the flow over each arc times the capacity of the arc
    '''
    #iterates over flow (i) and sink node (j) in the set of arcs
    return pyo.summation(model.f[i,j] for (i, j) in model.A if j == pyo.value(model.t))
#sets the objective function to maximize the total flow into the sink node
model.total = pyo.Objective(rule=total_rule, sense=pyo.maximize)

### Constraints

Enforce an upper limit on the flow across each arc  
 $f_{i,j} \leq c_{i,j}$, $\forall (i,j) \in A$

Enforce flow through each node  
 $\sum_{\{i \mid (i,j) \in A\}} f_{i,j} = \sum_{\{i \mid (j,i) \in A\}} f_{j,i}$, $\forall j \in N - \{s,t\}$
 
Flow lower bound  
 $f_{i,j} \geq 0$, $\forall (i,j) \in A$

Similarly, rule functions are used to define constraint expressions in the `Constraint` component:

In [ ]:
# Enforce an upper limit on the flow across each arc
#the flow over each arc should be less than or equal to the capacity of the arc
def limit_rule(model, i, j):
    '''
    Function to limit the flow over each arc to the capacity of the arc
    
    Parameters
    ----------
    model (AbstractModel): The model for which the constraint is applied
    i (int): The source node of the arc
    j (int): The sink node of the arc
    
    Returns
    -------
    inequality: The constraint that the flow over each arc should be less than or equal to the capacity of the arc
    '''
    return model.f[i,j] <= model.c[i, j]
#sets constraint that all arcs are following limit rule
model.limit = pyo.Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
#flow into a node should be equal to the flow out of the node
def flow_rule(model, k):
    '''
    Function to enforce flow through each node constraint
    
    Parameters
    ----------
    model (AbstractModel): The model for which the constraint is applied
    k (int): The node in the network
    '''
    if k == pyo.value(model.s) or k == pyo.value(model.t):
        return pyo.Constraint.Skip
    inFlow  = pyo.summation(model.f[i,j] for (i,j) in model.A if j == k)
    outFlow = pyo.summation(model.f[i,j] for (i,j) in model.A if i == k)
    #flow into a node(from a source node) should be equal to the flow out of the node (to a sink node)
    return inFlow == outFlow
#sets constraint that all nodes are following flow rule
model.flow = pyo.Constraint(model.N, rule=flow_rule)

## Model Data

Since this is an abstract Pyomo model, the set and parameter values need to be provided to initialize the model.  The following data command file provides a synthetic data set:

In [8]:
!cat maxflow.dat

set N := Zoo A B C D E Home;
set A := (Zoo,A) (Zoo,B) (A,C) (A,D) (B,A) (B,C) (C,D) (C,E) (D,E) (D,Home) (E,Home);

param s := Zoo;
param t := Home;
param: c :=
Zoo A 11
Zoo B 8
A C 5
A D 8
B A 4
B C 3
C D 2
C E 4
D E 5
D Home 8
E Home 6;


Set data is defined with the `set` command, and parameter data is defined with the `param` command.

This data set considers the problem of maximizing flow in a zoo. A panda is about to give birth at the zoo! Officials anticipate that attendance will skyrocket to see the new, adorable baby panda. There's one particular residential area called "Home" that is full of panda loving families and there's a fear that the increased number of people visiting the zoo will overload the public transportation system. It will be especially bad in the evening since the zoo closes about the same time as rush hour, so everyone will be trying to find space on the already crowded buses and subways. As a city planner, you were given a map of routes from the zoo to Home, along with the estimated number of families that could go on each route. Additionally, it was estimated that 16 families from Home will visit each day, and it's your task to figure out if this will overload the public transportation system, and, if it does, how could the system be improved?

## Solution

Pyomo includes a `pyomo` command that automates the construction and optimization of models.  The GLPK solver can be used in this simple example:

Install the MIP solver GLPK and then solve the model.

In [9]:
# Let's install the LP/MIP solver GLPK
if IN_COLAB:
    !apt-get install -y -qq glpk-utils

# Define the solver GLPK
if IN_COLAB:
    opt_glpk = SolverFactory('glpk', executable='/usr/bin/glpsol')
else:
    opt_glpk = SolverFactory('glpk')
# Here we could use another solver, e.g. gurobi or cplex
# opt_gurobi = SolverFactory('gurobi')

#creating a model instance for the abstract model using the data in maxflow.dat
instance = model.create_instance('maxflow.dat')

# We obtain the solution of model instance with GLPK
result_obj = opt_glpk.solve(instance, tee=False)
instance.display()

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.02] Applying solver
[    0.06] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 14.0
    Solver results file: results.json
[    0.06] Applying Pyomo postprocessing actions
[    0.06] Pyomo Finished


This output tells us how many people should travel along each route for the optimal solution. More importantly, though, is the line which says our objective value is 14. This means that at most 14 families can arrive at Home. However, we were told 16 families from Home were expected to visit the zoo each day. Therefore, unless something is done, the public transportation network in place will be overloaded.

## References

* A. Schrijver, (2002). "On the history of the transportation and maximum flow problems". Mathematical Programming 91 (3): 437–445. 